# El Modelo de Ramsey Estocástico con Programación Dinámica Estocástica y shock a la MIT
Profesor: PhD Carlos J. García 

Estudiante: David Limpe Cruz

MAE  - Universidad Alberto Hurtado - 2023

### El Problema de Opmización 

El problema de optimización del planificador central es el siguiente:

\begin{array}{rcl}

\max_{C_t, K_{t+1}} & = & E_t \sum_{t=0}^{\infty}\beta^{t}\ln C_{t} \\
s.a &  & C_{t}+I_{t}=S_t K_{t}^{\alpha}\\
 &  & k_{t+1}=I_{t}+(1-\delta)K_{t}\\
 &  & K_{0},S_0 \ dados.
\end{array}


Alternativamente:

\begin{array}{rcl}
\max_{C_t,K_{t+1}} & = & E_t \sum_{t=0}^{\infty}\beta^{t}\ln C_{t}\\
s.a &  & C_{t}+K_{t+1}=S_t K_{t}^{\alpha}+(1-\delta)K_{t}\\
 &  & K_{0},S_0 \ dados.

\end{array}

Por las condiciones iniciales, podemos asumir solución interior.

En términos de programación dinámica:

La ecuación de Bellman es:

$$
V(K_{t})=\max_{C_{t}, K_{t+1}}\left\{\log(C_t)+\beta E_tV(K_{t+1})\right\} 
$$

La solución del problema funcional está dada por la función valor y la función de política:

$$
V(K_{t})\,\,y\,\,K_{t+1}=g(K_{t})
$$

## Condiciones de Optimalidad

El lagranjeano intertemporal en valor presente:

$$L=\max_{C_t,K_{t+1}} E_t\sum_{t=0}^{\infty}\beta^{t}\left[\ln C_{t}+\lambda_{t}\left(S_{t}K_{t}^{\alpha}+(1-\delta)K_{t}-C_{t}-K_{t+1}\right)\right]$$

Condiciones de primer orden:

\begin{array}{rcl}
\frac{\partial L}{\partial C_{t}} & : & \beta^{t}\left[\frac{1}{C_{t}}-\lambda_{t}\right]=0\\

\frac{\partial L}{\partial K_{t+1}} & : & -\beta^{t}\lambda_{t}+\beta^{t+1}E_{t}\left[\lambda_{t+1}\left(\alpha S_{t+1}K_{t+1}^{\alpha-1}+1-\delta\right)\right]\\

\frac{\partial L}{\partial\lambda_{t}} & : & S_{t}K_{t}^{\alpha}+(1-\delta)K_{t}-C_{t}-K_{t+1}=0
\end{array}\\

Tenemos el siguiente sistema de ecuaciones en diferencias (no lineal):

\begin{array}{rcl}
\frac{1}{C_{t}} & = & \beta E_{t}\left[\frac{1}{C_{t+1}}\left(\alpha S_{t+1}K_{t+1}^{\alpha-1}+1-\delta\right)\right]\\

C_{t}+K_{t+1} & = & S_{t}K_{t}^{\alpha}+(1-\delta)K_{t}\\

\end{array}

## Estado Estacionario

Entonces el estado estacionario resuelve, asumiendo $t= t+1$:

\begin{array}{rcl}
 \frac{1}{\beta} & = & \left[\alpha SK^{*\alpha-1}+1-\delta\right]\\

 K^{*} & = & [\frac{r+\delta}{\alpha}]^{\frac{1}{\alpha-1}} \\
 

C^{*} & = & K^{*\alpha}-\delta K^{*}
\end{array}

### Implementación de la Solución Numérica

Algoritmo:

1. Definir un grid de puntos $k$ (discretización) y realizar una conjetura para $V(K)$ en cada posible punto del grid. 
2. Usando la conjetura evaluar el operador $Tv$ para cada punto en el grid (esto implica resolver el $\max$).
3. Evaluar si $v=Tv$ (usando el grado de tolerancia definido en parámetros), en tal caso terminar el proceso. Caso contrario volver al punto 2 usando la función valor resultante.

En términos de la ecuación de Bellman, la iteración sería:
$$
V_{j+1}(K)=\max_{K'}\left\{ \ln\left(SK^{\alpha}+(1-\delta)K-K'\right)+\beta E[V_{j}(K')]\right\} 
$$

Partir con algún $V_{0}(K)$. 

Para el cálculo del estado estacionario definimos primero los parámetros del modelo:

Parametros del modelo:

In [1]:
#Parámetros del modelo
beta = 0.98 #factor de descuento intertemporal
delta = 0.2 #depreciación del capital
alpha = 1/3 
r = (1/beta) - 1 #tasa de interés real
ctiny = 6.3829e-4 
crit = 1          # Criterio de convergencia          
tol = 0.0001 #tolerancia para determinar la convergencia        


0.0001

Definimos el grid de puntos de $k$ como $k_1,...,k_n \in [k_{min},k_{max}]$ 

In [2]:
#Stock de capital estacionario y definición de la malla o krid de puntos de k
k0 = ((1/alpha)*(r + delta))^(1/(alpha - 1))    #Es el estado estacionario del Capital
kpoints = 1000                                  # Puntos del krid
dev = 0.9                                       # Desviación en torno al Estado Estacionario  
kmin = k0 * (1 - dev)                          
kmax = k0 * (1 + dev) 
dk = (kmax - kmin) / (kpoints - 1)              # Calcula la tasa de incremento
k = kmin:dk:kmax                                # Distintos valores para el capital  
k = collect(k)              # vector columna de los krid de puntos de k con tasa de crecimiento positiva
println("k de estado estacionario: $k0")

k de estado estacionario: 1.8598443714639123


Note que mientras más puntos tengamos en el grid, más precisión tendremos.

El siguiente paso es iterar la función valor partiendo de la siguiente conjetura:
$$
v_{0}(K)=\left[\begin{array}{cc}
0 \\
\vdots \\
0 
\end{array}\right]
$$

In [3]:
# Definir valores iniciales
tv=zeros(size(k))              #vector de ceros de tamaño 1000*1
println(tv)
indice_i = zeros(size(k));
indice_i = floor.(Int, indice_i);

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

Resolvemos usando el método de iteración de la función valor (VFI):

In [4]:
#Adivinanza funcion de valor
v = zeros(size(k)); #es un vector {Float64} igual que tv, debe ser asi para que se puedan restar en el error_i 

Definimos las posibilidades de consumo y la función de Utilidad

In [6]:
#Posibilidades de consumo
c = (((k.^alpha) + (1 - delta)*k)*ones(size(k'))) - ones(size(k))*k' # de la Restricción de recursos
# es una matriz de 1000*1000, el cual genera todas las posibilidades de consumo.
c[c.==0].=ctiny;                     # Saca los consumos negativos
c[c.<=0].=ctiny/1000000000000000000000000   # Saca los consumos pequeños, por la función logaritmica
u = log.(c[:,:]);                           
#es una matriz que genera la Funcion de utilidad con todas las posibilidad

Aplicación del algoritmo del Contraction Mapping Theorem

In [7]:
#aplicación del CMT
j = 1 #para ver en cuantas iteraciones converge
while crit > tol
    for i=1:kpoints
        tv[i],indice_i[i] = findmax(u[i,:] + beta*v) #findmax entrega el maximo valor y su indice (posición)
        end    
    error_i=abs.(tv - v)
    crit = maximum(error_i)                
    copyto!(v,tv)
   global v 
   global crit 
   global j += 1    
end

Solución numérica

In [8]:
# Solución Final Numérica

copt  = k.^alpha + (1 - delta)*k - k[indice_i]   #calculo numérico
kk_0  = k[indice_i] 
b1    =   (log.(k[:,:]).- log(k0))\(log.(kk_0[:,:]).- log(k0)) 
b11   =   b1[1,1] 
typeof(b11)

kkk_0 = exp.(b11*(log.(k[:,:]).- log(k0)).+log(k0)) 

copte = k.^alpha + (1 - delta)*k - kkk_0;

### Gráficos de las funciones de política

Capital y Consumo óptimo

In [9]:
using Plots
nombre_vars = ["Linea de 45 grados" "Capital óptimo" "Consumo óptimo"]
plot1 = plot(k, [k kk_0 copt], xlabel="Capital", ylabel="Consumo", label=nombre_vars, title="Consumo óptimo")
savefig(plot1, "cons_opt.png") 

"d:\\@MAE_UAH_2023_ciclo II oficial\\Macro Internacional\\códigos_Julia_FI\\cons_opt.png"

### Shock a la MIT

Realizamos un shock 1,5 veces el estado estacionario en el modelo de Ramsey

In [14]:
## Simulación Shock MIT
N        = 20;
kk       = zeros(N,1)
AA       = zeros(N,1)
kk[1]  = 1.5*k0
log(k0)
kkk      = zeros(N,1)
#kkk[1,:] .= exp(b11*(log(kk[1,;]). - log(k0)). + log(k0))
kkk[1]   = exp.(b11*(log(kk[1]) - log(k0)) + log(k0))
cosim    = zeros(N,1)
#cosim(1) = (kk(1)^alpha) + (1 - delta)*kk(1) - kkk(1);
cosim[1] = kk[1].^alpha + (1 - delta)*kk[1] - kkk[1]
AA[1]    = (cosim[1] + kkk[1] - (1 - delta)*kk[1])/(kk[1]^alpha)
   
for j=2:N
    
        kkk[j]   = exp.(b11*(log(kkk[j-1]) - log(k0)) + log(k0));
        cosim[j] = kkk[j-1]^alpha + (1 - delta)*kkk[j-1] - kkk[j];
        
 end

In [15]:
#Gráfico
t=1:1:N
nomb_vars1 = ["Consumo" "Capital"]
plt1= plot(t, [cosim kkk], ylabel ="Unidades de consumo y capital", xlabel="Tiempo", label=nomb_vars1, title="Shock a la MIT", linewidth = 2)
savefig(plt1, "IRF_MIT.png") 


"d:\\@MAE_UAH_2023_ciclo II oficial\\Macro Internacional\\códigos_Julia_FI\\IRF_MIT.png"